# Imoporting Packaging

In [1]:
# following: https://www.youtube.com/watch?v=_mlCswQeM4E

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [8]:
import json
import requests  # to communicate with the rest API
from socketIO_client import SocketIO

In [12]:
TRADING_API_URL = 'https://api-demo.fxcm.com:443'
WEBSOCKET_PORT = 443

ACCESS_TOKEN = os.environ.get('token_fxcm')

In [27]:
def on_connect():
    print('Websocket Connected: ', socketIO._engineIO_session.id)
    
def onclose():
    print('Websocket Close')

socketIO = SocketIO(TRADING_API_URL, WEBSOCKET_PORT, params={'access_token':ACCESS_TOKEN})

socketIO.on('connect',on_connect)
socketIO.on('disconnect',onclose)

bearer_access_token = "Bearer " + socketIO._engineIO_session.id + ACCESS_TOKEN

print(bearer_access_token)

Bearer 5wgx0VoxLcPr2Z72ABDEb4fa13be7de98859d19b2fcdbb81d3e40df2c534


# Pulling Historical Pricing

In [28]:
method = '/candles/1/H1'

hist_response = requests.get(TRADING_API_URL + method,
                            headers = {
                                'User-Agent':'request',
                                'Authorization': bearer_access_token,
                                'Accept':'application/json',
                                'Content-Type':'application/x-www-form-urlencoded'
                            },
                            params = {
                                'num':1000,
                                'from':1494086400,
                                'to':1503835200
                            })

In [29]:
print(hist_response)

<Response [200]>


In [30]:
print(hist_response.json())

{'response': {'executed': True, 'error': ''}, 'instrument_id': '1', 'period_id': 'H1', 'candles': [[1494183600, 1.09974, 1.1018, 1.1021, 1.10147, 1.10014, 1.10211, 1.10245, 1.10186, 922], [1494187200, 1.1018, 1.10157, 1.10217, 1.10135, 1.10211, 1.10232, 1.10249, 1.10176, 516], [1494190800, 1.10157, 1.10107, 1.10205, 1.10006, 1.10232, 1.10147, 1.10245, 1.10055, 4462], [1494194400, 1.10107, 1.09923, 1.10128, 1.09841, 1.10147, 1.09941, 1.10152, 1.09863, 9647], [1494198000, 1.09923, 1.0964, 1.09952, 1.09627, 1.09941, 1.09658, 1.09971, 1.09643, 8087], [1494201600, 1.0964, 1.09661, 1.09735, 1.09562, 1.09658, 1.09677, 1.0975, 1.0958, 9961], [1494205200, 1.09661, 1.0974, 1.09757, 1.09627, 1.09677, 1.09754, 1.09773, 1.09645, 7731], [1494208800, 1.0974, 1.09746, 1.09819, 1.09709, 1.09754, 1.09761, 1.09836, 1.09726, 5127], [1494212400, 1.09746, 1.09706, 1.09751, 1.09648, 1.09761, 1.09722, 1.09768, 1.09665, 4601], [1494216000, 1.09706, 1.09716, 1.09722, 1.09603, 1.09722, 1.09732, 1.0974, 1.09621, 

In [36]:
if hist_response.status_code==200:
    print('Data retrieved')
    
    hist_data = hist_response.json()
    candle_data = hist_data['candles']
    
    df = pd.DataFrame(candle_data)
    df.columns = ['time','bidopen','bidclose','bidhigh','bidlow','askopen','askclose',
                 'askhigh','asklow','TikQty']
    df['time'] = pd.to_datetime(df['time'],unit='s')
else:
    print(hist_response)
print(df.shape)
df.head(5)

Data retrieved
(1952, 10)


,time,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,TikQty
0,2017-05-07 19:00:00,1.09974,1.10180,1.10210,1.10147,1.10014,1.10211,1.10245,1.10186,922
1,2017-05-07 20:00:00,1.10180,1.10157,1.10217,1.10135,1.10211,1.10232,1.10249,1.10176,516
2,2017-05-07 21:00:00,1.10157,1.10107,1.10205,1.10006,1.10232,1.10147,1.10245,1.10055,4462
3,2017-05-07 22:00:00,1.10107,1.09923,1.10128,1.09841,1.10147,1.09941,1.10152,1.09863,9647
4,2017-05-07 23:00:00,1.09923,1.09640,1.09952,1.09627,1.09941,1.09658,1.09971,1.09643,8087


# Streaming Real time prices

In [43]:
method = '/subscribe'

sub_response = requests.post(TRADING_API_URL + method,
                            headers = {
                                'User-Agent':'request',
                                'Authorization': bearer_access_token,
                                'Accept':'application/json',
                                'Content-Type':'application/x-www-form-urlencoded'
                            },
                            data = {
                                'pairs':'EUR/USD'
                            })
print(sub_response)
print(sub_response.json())

<Response [200]>
{'response': {'executed': False, 'error': 'Cannot subscribe to symbol: USD/EUR'}, 'pairs': []}


In [ ]:
# But the ideal is to get the updates as soon as they are available
def on_price_update(msg):
    response = json.loads(msg)
    print(response)
    
if sub_response.status_code == 200:
    socketIO.on('EUR/USD',on_price_update)
    socketIO.wait()